In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, split, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import Tokenizer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml.feature import CountVectorizerModel
from pyspark.ml.classification import NaiveBayesModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import when, col
from pyspark.ml import PipelineModel




In [ ]:
!pip install pyspark


In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json when prompted


Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"saiharshithakomati","key":"a6b19059ae067acc34430ea3d09b2e85"}'}

In [ ]:
import os

# Rename the uploaded file to kaggle.json
os.rename("kaggle (2).json", "kaggle.json")


In [ ]:
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list


ref                                                         title                                                   size  lastUpdated                 downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
atharvasoundankar/chocolate-sales                           Chocolate Sales Data 📊🍫                                14473  2025-03-19 03:51:40.270000          18736        314  1.0              
adilshamim8/student-depression-dataset                      Student Depression Dataset                            467020  2025-03-13 03:12:30.423000           9699        139  1.0              
atharvasoundankar/impact-of-ai-on-digital-media-2020-2025   🌍 Impact of AI on Digital Media (2020-2025)             5812  2025-04-03 09:12:25.070000           1075         26  1.0              
ertugrulesol/online-retail-dat

In [ ]:
!kaggle datasets download -d nikhileswarkomati/suicide-watch
!unzip suicide-watch.zip


Dataset URL: https://www.kaggle.com/datasets/nikhileswarkomati/suicide-watch
License(s): CC-BY-SA-4.0
Archive:  suicide-watch.zip
  inflating: Suicide_Detection.csv   


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/Suicide_Detection.csv")

# Show dataset info
df.info()

# Display first few rows
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232074 entries, 0 to 232073
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  232074 non-null  int64 
 1   text        232074 non-null  object
 2   class       232074 non-null  object
dtypes: int64(1), object(2)
memory usage: 5.3+ MB


,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [ ]:
# Drop 'Unnamed: 0' column (not useful)
df = df.drop(columns=['Unnamed: 0'])

# Show dataset info after dropping
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232074 entries, 0 to 232073
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    232074 non-null  object
 1   class   232074 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: Load the correct dataset
#df = pd.read_csv("/content/Suicide_Detection.csv")

# Step 2: Drop the 'Unnamed: 0' column (index column from CSV)
#df.drop(columns=["Unnamed: 0"], inplace=True)

# Step 3: Rename 'class' to 'label' for consistency
df.rename(columns={"class": "label"}, inplace=True)

# Step 4: Split 70% train, 15% dev, 15% test
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
dev_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

# Step 5: Save to Drive
train_df.to_csv("/content/drive/MyDrive/final_train.csv", index=False)
dev_df.to_csv("/content/drive/MyDrive/final_dev.csv", index=False)
test_df.to_csv("/content/drive/MyDrive/final_test.csv", index=False)

# Step 6: Preview first few rows
print("✅ Train Set:")
print(train_df.head())
print("\n✅ Dev Set:")
print(dev_df.head())
print("\n✅ Test Set:")
print(test_df.head())


✅ Train Set:
                                                     text        label
58216   No guardian angelI met my partner back in 2012...      suicide
1052    i'm losing all hope. it's not getting better. ...      suicide
177925  The thotery is increasing The skirt trend made...  non-suicide
228923       Every time we age Our parents rage also ages  non-suicide
16486   My friend is suicidal and I'm afraidMy friend ...      suicide

✅ Dev Set:
                                                     text        label
57051   Seriously considering suicideIt’s the middle o...      suicide
125426  Why tf is she still such a huge part of my lif...  non-suicide
78169   I keep having dreams about my old friends I do...  non-suicide
118469  I am close to ending it all, How should it be ...      suicide
105608  I don't get itI am not even very sad right now...      suicide

✅ Test Set:
                                                     text        label
212177  I woke up yesterday with the fe

In [ ]:
# Load data
df = pd.read_csv("/content/drive/MyDrive/final_train.csv")

In [ ]:
df.sample(n=20)

,text,label
116181,Can someone help me find a good book For Engli...,non-suicide
52085,Can you help?!Hi guys!\nI know it's a long sho...,suicide
36556,Posting every day until 2021 - day 83 Nothing ...,non-suicide
97505,I've picked a dateI made my fourth but most se...,suicide
99653,I could have been beautifulI have low function...,suicide
57191,Those of you in relationships...what's it like...,non-suicide
99882,I've been wondering what does a bag of Freezin...,non-suicide
23472,I overdosed yesterdayIt wasn't enough and I co...,suicide
108071,This is too much.I dont know why I am even pos...,suicide
2960,Hello thereI will absolutely die from suicide....,suicide


In [ ]:
import pandas as pd
from sklearn.utils import shuffle

# Load data
df = pd.read_csv("/content/drive/MyDrive/final_train.csv")

# Drop nulls
df = df.dropna(subset=["text", "label"])

# Filter suicide and non-suicide
suicidal_df = df[df["label"].str.lower() == "suicide"]
non_suicidal_df = df[df["label"].str.lower() != "suicide"]

# Print lengths to be sure
print(f"Suicide: {len(suicidal_df)}, Non-Suicide: {len(non_suicidal_df)}")

# Ensure we sample only as much as exists
min_len = min(len(suicidal_df), len(non_suicidal_df))
suicidal_sample = suicidal_df.sample(n=min_len, random_state=42)
non_suicidal_sample = non_suicidal_df.sample(n=min_len, random_state=42)

# Combine and shuffle
balanced_df = pd.concat([suicidal_sample, non_suicidal_sample])
balanced_df = shuffle(balanced_df, random_state=42)

# Save it
balanced_df.to_csv("/content/drive/MyDrive/balanced_final_train.csv", index=False)

print("✅ Balanced dataset saved as 'balanced_final_train.csv'")


Suicide: 81226, Non-Suicide: 81225
✅ Balanced dataset saved as 'balanced_final_train.csv'


In [ ]:
len(df)


162451

In [ ]:
df.sample(n=20)


,text,label
117449,hiii how was everyones day? i think mine was a...,0
79656,"Lonely.I know this is cringy as fuck to say, b...",1
35553,I'm really at the end of my rope.Today I have ...,1
110487,IF i had a dollar I would have 3.57 soles \ni...,0
73575,I've been talking to this girl. She's in my se...,0
136009,Found a really cute redditor. [This](https://w...,0
10777,Some useless rant about subreddits I hate it w...,0
86417,"Being told that I ""overshare too much"" made me...",1
4625,Apparently California executed 2 people by gas...,0
137338,Monday MorningI think I've decided to go on Mo...,1


In [ ]:
df['label'].nunique()


2

In [ ]:
print("Total rows:", len(df))
print(df["label"].value_counts(dropna=False))


Total rows: 162451
label
suicide        81226
non-suicide    81225
Name: count, dtype: int64


In [ ]:
# Step 1: Remove nulls
df = df.dropna(subset=["text", "label"])

# Step 2: Convert labels
df["label"] = df["label"].apply(lambda x: 1 if x == "suicide" or x == 1 else 0)

# Step 3: Check counts
print(df["label"].value_counts())


label
1    81226
0    81225
Name: count, dtype: int64


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# Start session
spark = SparkSession.builder.appName("FixCSVLoad").getOrCreate()

# Define proper schema
schema = StructType([
    StructField("text", StringType(), True),
    StructField("label", StringType(), True)
])

# Read the CSV properly
train_df = spark.read.option("header", True) \
    .option("multiLine", True) \
    .option("quote", '"') \
    .option("escape", '"') \
    .schema(schema) \
    .csv("/content/drive/MyDrive/final_train.csv")

# Check first few rows and schema
train_df.printSchema()
train_df.show(5, truncate=False)


root
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
train_df.select("label").distinct().show(truncate=False)


+-----------+
|label      |
+-----------+
|suicide    |
|non-suicide|
+-----------+



Preprocesing

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, when
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

# 3. Drop null values
train_df = train_df.dropna(subset=["text", "label"])

# 4. Convert labels to binary
train_df = train_df.withColumn("label", when(col("label") == "suicide", 1).otherwise(0))

# 5. Clean the text
train_df = train_df.withColumn("text", lower(col("text")))
train_df = train_df.withColumn("text", regexp_replace("text", "[^a-z\\s]", ""))


In [ ]:
train_df.groupBy("label").count().show(truncate=False)


+-----+-----+
|label|count|
+-----+-----+
|1    |81226|
|0    |81225|
+-----+-----+



Random Forest

In [ ]:

# 6. Define the stages of the pipeline
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=10000)
idf = IDF(inputCol="raw_features", outputCol="features")

# 7. Random Forest Classifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=200, maxDepth=10)

# 8. Build the pipeline
pipeline_rf = Pipeline(stages=[tokenizer, remover, hashingTF, idf, rf])

# 9. Train the model
model_rf = pipeline_rf.fit(train_df)

# 10. Save the trained model
model_rf.write().overwrite().save("/content/drive/MyDrive/pipeline_rf_model_final")

print("Random Forest pipeline trained and saved successfully!")


Random Forest pipeline trained and saved successfully!


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, when
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF

# 1. Start Spark session
#spark = SparkSession.builder.appName("LoadDevData").getOrCreate()

# 2. Define schema (same as train)
schema = StructType([
    StructField("text", StringType(), True),
    StructField("label", StringType(), True)
])

# 3. Load the dev CSV properly
dev_df = spark.read.option("header", True) \
    .option("multiLine", True) \
    .option("quote", '"') \
    .option("escape", '"') \
    .schema(schema) \
    .csv("/content/drive/MyDrive/final_dev.csv")   # <-- replace with actual path if different

# 4. Print schema and unique labels
dev_df.printSchema()
dev_df.select("label").distinct().show(truncate=False)



root
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)

+-----------+
|label      |
+-----------+
|suicide    |
|non-suicide|
+-----------+



In [ ]:
# 5. Drop nulls
dev_df = dev_df.dropna(subset=["text", "label"])

# 6. Convert labels to binary
dev_df = dev_df.withColumn("label", when(col("label") == "suicide", 1).otherwise(0))

# 7. Text cleaning
dev_df = dev_df.withColumn("text", lower(col("text")))
dev_df = dev_df.withColumn("text", regexp_replace("text", "[^a-z\\s]", ""))

# 8. Confirm class balance after preprocessing
dev_df.groupBy("label").count().show()


+-----+-----+
|label|count|
+-----+-----+
|    1|17405|
|    0|17406|
+-----+-----+



In [ ]:
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 9. Load the saved Random Forest pipeline model
loaded_rf_model = PipelineModel.load("/content/drive/MyDrive/pipeline_rf_model_final")

# 10. Run prediction on dev set
predictions = loaded_rf_model.transform(dev_df)

# 11. See some sample predictions
predictions.select("text", "label", "prediction").show(10, truncate=False)

# 12. Evaluate with metrics
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

accuracy = evaluator_accuracy.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)
f1 = evaluator_f1.evaluate(predictions)

# 13. Display metrics
print(f"✅ Accuracy:  {accuracy * 100:.2f}%")
print(f"✅ Precision: {precision * 100:.2f}%")
print(f"✅ Recall:    {recall * 100:.2f}%")
print(f"✅ F1 Score:  {f1 * 100:.2f}%")


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Bigram_RandomForest model

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, NGram, HashingTF, IDF
from pyspark.ml.classification import RandomForestClassifier

# Pipeline stages
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
bigrams = NGram(n=2, inputCol="filtered_words", outputCol="bigrams")
hashingTF = HashingTF(inputCol="bigrams", outputCol="raw_features", numFeatures=20000)
idf = IDF(inputCol="raw_features", outputCol="features")

rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=200, maxDepth=10)

pipeline_bigram_rf = Pipeline(stages=[tokenizer, remover, bigrams, hashingTF, idf, rf])


In [ ]:
model_bigram_rf = pipeline_bigram_rf.fit(train_df)
model_bigram_rf.write().overwrite().save("/content/drive/MyDrive/pipeline_bigram_rf_model")


In [ ]:
predictions = model_bigram_rf.transform(dev_df)


In [ ]:
# 9. Load the saved Random Forest pipeline model
loaded_rf_model_bi = PipelineModel.load("/content/drive/MyDrive/pipeline_bigram_rf_model")

# 10. Run prediction on dev set
predictions = loaded_rf_model_bi.transform(dev_df)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision").evaluate(predictions)
recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall").evaluate(predictions)
f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1").evaluate(predictions)

print(f"✅ Accuracy:  {accuracy * 100:.2f}%")
print(f"✅ Precision: {precision * 100:.2f}%")
print(f"✅ Recall:    {recall * 100:.2f}%")
print(f"✅ F1 Score:  {f1 * 100:.2f}%")

✅ Accuracy:  77.61%
✅ Precision: 80.10%
✅ Recall:    77.61%
✅ F1 Score:  77.14%


In [ ]:
train_df.groupBy("label").count().show(truncate=False)


+-----+-----+
|label|count|
+-----+-----+
|1    |81226|
|0    |81225|
+-----+-----+



GBTClassifier model

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, when
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Pipeline

# 5. TF-IDF pipeline setup
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=10000)
idf = IDF(inputCol="raw_features", outputCol="features")

# 6. GBTClassifier
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=50, maxDepth=5)

# 7. Full pipeline
pipeline_gbt = Pipeline(stages=[tokenizer, remover, hashingTF, idf, gbt])

# 8. Train
model_gbt = pipeline_gbt.fit(train_df)

# 9. Save model
model_gbt.write().overwrite().save("/content/drive/MyDrive/pipeline_gbt_final")

print("✅ GBTClassifier model trained and saved successfully.")

✅ GBTClassifier model trained and saved successfully.


In [ ]:
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 9. Load the saved Random Forest pipeline model
loaded_rf_model = PipelineModel.load("/content/drive/MyDrive/pipeline_gbt_final")

# 10. Run prediction on dev set
predictions = loaded_rf_model.transform(dev_df)

# 11. See some sample predictions
predictions.select("text", "label", "prediction").show(10, truncate=False)

# 12. Evaluate with metrics
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

accuracy = evaluator_accuracy.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)
f1 = evaluator_f1.evaluate(predictions)

# 13. Display metrics
print(f"✅ Accuracy:  {accuracy * 100:.2f}%")
print(f"✅ Precision: {precision * 100:.2f}%")
print(f"✅ Recall:    {recall * 100:.2f}%")
print(f"✅ F1 Score:  {f1 * 100:.2f}%")


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

optimise GBTClassifier model

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, when
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Pipeline

# Feature extraction
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="raw_features", vocabSize=5000, minDF=5)
idf = IDF(inputCol="raw_features", outputCol="features")

# GBTClassifier with optimized params
gbt = GBTClassifier(featuresCol="features", labelCol="label", maxIter=20, maxDepth=4)

# Build pipeline
pipeline_gbt = Pipeline(stages=[tokenizer, remover, vectorizer, idf, gbt])

# Train model
model_gbt = pipeline_gbt.fit(train_df)

# Save model
model_gbt.write().overwrite().save("/content/drive/MyDrive/pipeline_gbt_optimized")

print("✅ GBTClassifier (optimized) trained and saved.")


✅ GBTClassifier (optimized) trained and saved.


In [ ]:
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 9. Load the saved Random Forest pipeline model
loaded_rf_model = PipelineModel.load("/content/drive/MyDrive/pipeline_gbt_optimized")

# 10. Run prediction on dev set
predictions = loaded_rf_model.transform(dev_df)

# 11. See some sample predictions
predictions.select("text", "label", "prediction").show(10, truncate=False)

# 12. Evaluate with metrics
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

accuracy = evaluator_accuracy.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)
f1 = evaluator_f1.evaluate(predictions)

# ✅ 13. Display metrics
print(f"✅ Accuracy:  {accuracy * 100:.2f}%")
print(f"✅ Precision: {precision * 100:.2f}%")
print(f"✅ Recall:    {recall * 100:.2f}%")
print(f"✅ F1 Score:  {f1 * 100:.2f}%")


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Naive Bayes Model

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, when
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="raw_features", vocabSize=5000, minDF=5)
idf = IDF(inputCol="raw_features", outputCol="features")

# 6. Naive Bayes Classifier (good for text classification)
nb = NaiveBayes(labelCol="label", featuresCol="features", modelType="multinomial")

# 7. Build the pipeline
pipeline_nb = Pipeline(stages=[tokenizer, remover, vectorizer, idf, nb])

# 8. Train the model
model_nb = pipeline_nb.fit(train_df)

# 9. Save the model
model_nb.write().overwrite().save("/content/drive/MyDrive/pipeline_nb_model")

print("✅ Naive Bayes pipeline trained and saved successfully!")

✅ Naive Bayes pipeline trained and saved successfully!


In [ ]:
# 9. Load the saved Random Forest pipeline model
loaded_rf_model = PipelineModel.load("/content/drive/MyDrive/pipeline_nb_model")

# 10. Run prediction on dev set
predictions = loaded_rf_model.transform(dev_df)

# 11. See some sample predictions
predictions.select("text", "label", "prediction").show(10, truncate=False)

# 12. Evaluate with metrics
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

accuracy = evaluator_accuracy.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)
f1 = evaluator_f1.evaluate(predictions)

# ✅ 13. Display metrics
print(f"✅ Accuracy:  {accuracy * 100:.2f}%")
print(f"✅ Precision: {precision * 100:.2f}%")
print(f"✅ Recall:    {recall * 100:.2f}%")
print(f"✅ F1 Score:  {f1 * 100:.2f}%")


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, lower, regexp_replace, when

# ✅ 1. Start Spark session (if not already started)
spark = SparkSession.builder.appName("LoadTestData").getOrCreate()

# ✅ 2. Define schema for test data
schema = StructType([
    StructField("text", StringType(), True),
    StructField("label", StringType(), True)
])

# ✅ 3. Load the test CSV properly
test_df = spark.read.option("header", True) \
    .option("multiLine", True) \
    .option("quote", '"') \
    .option("escape", '"') \
    .schema(schema) \
    .csv("/content/drive/MyDrive/final_test.csv")

# ✅ 4. Check schema and label values
test_df.printSchema()
test_df.select("label").distinct().show(truncate=False)


root
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)

+-----------+
|label      |
+-----------+
|suicide    |
|non-suicide|
+-----------+



In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import lower, regexp_replace, when
# ✅ Clean CSV formatting

test_df = test_df.dropna(subset=["text", "label"])
test_df = test_df.withColumn("label", when(col("label") == "suicide", 1).otherwise(0))
test_df = test_df.withColumn("text", lower(col("text")))
test_df = test_df.withColumn("text", regexp_replace("text", "[^a-z\\s]", ""))



In [ ]:
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# === Step 1: Load All 3 Saved Pipeline Models ===
pipeline_nb = PipelineModel.load("/content/drive/MyDrive/pipeline_nb_model")
pipeline_rf = PipelineModel.load("/content/drive/MyDrive/pipeline_rf_model_final")
pipeline_gbt = PipelineModel.load("/content/drive/MyDrive/pipeline_gbt_final")


# === Step 2: Define Evaluation Function ===
def evaluate_model(pred_df, model_name):
    evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
    evaluator_prec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
    evaluator_rec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")

    accuracy = evaluator_acc.evaluate(pred_df)
    f1 = evaluator_f1.evaluate(pred_df)
    precision = evaluator_prec.evaluate(pred_df)
    recall = evaluator_rec.evaluate(pred_df)

    print(f"\n🔍 Results for {model_name}")
    print(f"Accuracy:  {round(accuracy * 100, 2)}%")
    print(f"F1 Score:  {round(f1 * 100, 2)}%")
    print(f"Precision: {round(precision * 100, 2)}%")
    print(f"Recall:    {round(recall * 100, 2)}%")

# === Step 3: Run Predictions ===
pred_nb = pipeline_nb.transform(test_df)
pred_rf = pipeline_rf.transform(test_df)
pred_gbt = pipeline_gbt.transform(test_df)

# === Step 4: Evaluate All ===
evaluate_model(pred_nb, "Naive Bayes")
evaluate_model(pred_rf, "Random Forest")
evaluate_model(pred_gbt, "GBT Classifier")

# === Optional: Show sample predictions for each ===
print("\n🧠 Sample Predictions for Naive Bayes:")
pred_nb.select("text", "label", "prediction").show(3, truncate=False)

print("\n🌲 Sample Predictions for Random Forest:")
pred_rf.select("text", "label", "prediction").show(3, truncate=False)

print("\n⚡ Sample Predictions for GBT Classifier:")
pred_gbt.select("text", "label", "prediction").show(3, truncate=False)



🔍 Results for Naive Bayes
Accuracy:  89.43%
F1 Score:  89.39%
Precision: 89.99%
Recall:    89.43%

🔍 Results for Random Forest
Accuracy:  82.83%
F1 Score:  82.74%
Precision: 83.57%
Recall:    82.83%

🔍 Results for GBT Classifier
Accuracy:  87.01%
F1 Score:  87.0%
Precision: 87.18%
Recall:    87.01%

🧠 Sample Predictions for Naive Bayes:
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

XGBoost

In [ ]:
# ✅ 1. Install
!pip install xgboost --quiet
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# ✅ 2. Import
import pandas as pd
import re
import joblib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# ✅ 3. Clean Function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    filtered = [w for w in words if w not in stop_words]
    return " ".join(filtered)

# ✅ 4. Load & Clean Train
train_df = pd.read_csv("/content/drive/MyDrive/final_train.csv").dropna(subset=["text", "label"])
train_df["text"] = train_df["text"].apply(clean_text)
train_df["label"] = train_df["label"].map({'non-suicide': 0, 'suicide': 1})

print("🟢 Train Sample:\n", train_df.head())
print("🔢 Train Label Counts:\n", train_df["label"].value_counts())

# ✅ 5. Load & Clean Dev
dev_df = pd.read_csv("/content/drive/MyDrive/final_dev.csv").dropna(subset=["text", "label"])
dev_df["text"] = dev_df["text"].apply(clean_text)
dev_df["label"] = dev_df["label"].map({'non-suicide': 0, 'suicide': 1})

print("\n🟣 Dev Sample:\n", dev_df.head())
print("🔢 Dev Label Counts:\n", dev_df["label"].value_counts())

# ✅ 6. Build & Train Pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(max_features=5000, ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', XGBClassifier(n_estimators=200, max_depth=5, use_label_encoder=False, eval_metric='logloss'))
])
pipeline.fit(train_df["text"], train_df["label"])
print("\n✅ Model trained successfully.")

# ✅ 7. Save the model
joblib.dump(pipeline, "/content/drive/MyDrive/xgb_pipeline.pkl")
print("💾 Model saved to Drive.")


🟢 Train Sample:
                                                 text  label
0  guardian angeli met partner back suicidal self...      1
1  im losing hope getting better quarantine ruine...      1
2   thotery increasing skirt trend made things worse      0
3              every time age parents rage also ages      0
4  friend suicidal im afraidmy friend depressed y...      1
🔢 Train Label Counts:
 label
1    81226
0    81225
Name: count, dtype: int64

🟣 Dev Sample:
                                                 text  label
0  seriously considering suicideits middle night ...      1
1  tf still huge part life liketheres girl reject...      0
2  keep dreams old friends dont know keeps happen...      0
3  close ending donei become useless person every...      1
4  dont get iti even sad right want still hate do...      1
🔢 Dev Label Counts:
 label
0    17406
1    17405
Name: count, dtype: int64


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:03:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



✅ Model trained successfully.
💾 Model saved to Drive.


In [ ]:
# ✅ 8. Load saved model & Predict on Dev
loaded_model = joblib.load("/content/drive/MyDrive/xgb_pipeline.pkl")
y_pred = loaded_model.predict(dev_df["text"])


In [ ]:
# ✅ 9. Evaluate
report_dict = classification_report(dev_df["label"], y_pred, output_dict=True)
accuracy = accuracy_score(dev_df["label"], y_pred)

print("\n📊 Evaluation Metrics (Dev Set):")
print(f"✅ Accuracy     : {round(accuracy * 100, 2)}%")
print(f"🔁 Precision    : {round(report_dict['weighted avg']['precision'] * 100, 2)}%")
print(f"🎯 Recall       : {round(report_dict['weighted avg']['recall'] * 100, 2)}%")
print(f"📏 F1-score     : {round(report_dict['weighted avg']['f1-score'] * 100, 2)}%")

print("\n🔍 Per-Class Metrics:")
for label in ['0', '1']:
    print(f"Class {label} → Precision: {report_dict[label]['precision']:.2f}, Recall: {report_dict[label]['recall']:.2f}, F1-score: {report_dict[label]['f1-score']:.2f}")



📊 Evaluation Metrics (Dev Set):
✅ Accuracy     : 91.95%
🔁 Precision    : 92.03%
🎯 Recall       : 91.95%
📏 F1-score     : 91.94%

🔍 Per-Class Metrics:
Class 0 → Precision: 0.90, Recall: 0.94, F1-score: 0.92
Class 1 → Precision: 0.94, Recall: 0.90, F1-score: 0.92
